# Phase 1: Basic Factors

Explore three fundamental trading signals on AAPL daily prices:
- **Momentum**: Price change over a lookback window
- **RSI**: Relative Strength Index (overbought/oversold oscillator)
- **MACD**: Moving Average Convergence Divergence (trend momentum)

For each factor, we compute the signal, backtest it, and compare risk metrics.


In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import pandas as pd

from qlib.backtesting import Backtester
from qlib.data import DataLoader
from qlib.factors import MACD, RSI, Momentum
from qlib.metrics import max_drawdown, sharpe, sortino

plt.style.use("seaborn-v0_8")

In [ ]:
symbol = "AAPL"
start = "2015-01-01"

print(f"Loading {symbol} data starting {start}...")
data = DataLoader.load(symbol, start=start)
data.head()

## Momentum

Momentum measures the percentage price change over a lookback window. Positive momentum suggests the trend may continue upward.

In [ ]:
lookback = 20
factor = Momentum(lookback=lookback)

signal = factor.compute(data)
print(f"Signal name: {signal.name or 'momentum'}; lookback={lookback}")
signal.tail()

In [ ]:
bt = Backtester(data["close"])
rets = bt.run(signal).dropna()

metrics = {
    "sharpe": sharpe(rets),
    "sortino": sortino(rets),
    "max_drawdown": max_drawdown(rets),
}
metrics

In [ ]:
cum_rets = (1 + rets).cumprod()
ax = cum_rets.plot(figsize=(10, 4), title="Momentum Strategy Cumulative Returns")
ax.set_ylabel("Growth of $1")
plt.show()

## RSI (Relative Strength Index)

RSI is a bounded oscillator (0-100) that measures whether recent gains outpace recent losses.
- **RSI > 70**: Potentially overbought (may reverse down)
- **RSI < 30**: Potentially oversold (may reverse up)

We'll use RSI as a contrarian signal: go long when oversold, short when overbought.

In [ ]:
rsi = RSI(period=14)
rsi_signal = rsi.compute(data)

# Convert RSI to a trading signal: -1 when overbought, +1 when oversold, 0 otherwise
rsi_position = pd.Series(0.0, index=rsi_signal.index)
rsi_position[rsi_signal > 70] = -1.0  # overbought -> short
rsi_position[rsi_signal < 30] = 1.0  # oversold -> long

print(f"RSI period: {rsi.period}")
rsi_signal.tail()

In [ ]:
bt_rsi = Backtester(data["close"])
rsi_rets = bt_rsi.run(rsi_position).dropna()

rsi_metrics = {
    "sharpe": sharpe(rsi_rets),
    "sortino": sortino(rsi_rets),
    "max_drawdown": max_drawdown(rsi_rets),
}
rsi_metrics

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

data["close"].plot(ax=axes[0], title="AAPL Price")
axes[0].set_ylabel("Price")

rsi_signal.plot(ax=axes[1], title="RSI (14)")
axes[1].axhline(70, color="red", linestyle="--", alpha=0.5, label="Overbought")
axes[1].axhline(30, color="green", linestyle="--", alpha=0.5, label="Oversold")
axes[1].set_ylabel("RSI")
axes[1].legend()

plt.tight_layout()
plt.show()

## MACD (Moving Average Convergence Divergence)

MACD measures momentum by comparing two EMAs of price. It produces three signals:
- **MACD Line**: Fast EMA minus Slow EMA
- **Signal Line**: EMA of the MACD line (smoothed)
- **Histogram**: MACD minus Signal (momentum of momentum)

We'll trade the histogram directly: positive = long, negative = short.

In [ ]:
macd = MACD(fast=12, slow=26, signal_period=9)
macd_result = macd.compute(data)

print(f"MACD params: fast={macd.fast}, slow={macd.slow}, signal={macd.signal_period}")
macd_result.to_frame().tail()

In [ ]:
bt_macd = Backtester(data["close"])
macd_rets = bt_macd.run(macd_result.histogram).dropna()

macd_metrics = {
    "sharpe": sharpe(macd_rets),
    "sortino": sortino(macd_rets),
    "max_drawdown": max_drawdown(macd_rets),
}
macd_metrics

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

data["close"].plot(ax=axes[0], title="AAPL Price")
axes[0].set_ylabel("Price")

macd_result.line.plot(ax=axes[1], label="MACD", color="blue")
macd_result.signal.plot(ax=axes[1], label="Signal", color="orange")
axes[1].axhline(0, color="gray", linestyle="--", alpha=0.5)
axes[1].set_title("MACD Line & Signal")
axes[1].legend()

# Color bars green (positive) or red (negative)
colors = ["green" if v >= 0 else "red" for v in macd_result.histogram]
axes[2].bar(
    macd_result.histogram.index,
    macd_result.histogram.values,
    color=colors,
    alpha=0.6,
    width=1,
)
axes[2].axhline(0, color="gray", linestyle="--", alpha=0.5)
axes[2].set_title("MACD Histogram")

plt.tight_layout()
plt.show()

## Comparison

Let's compare the three strategies side by side.


In [ ]:
comparison = pd.DataFrame(
    {
        "Momentum": metrics,
        "RSI": rsi_metrics,
        "MACD": macd_metrics,
    }
).T

comparison.round(3)

In [ ]:
# Cumulative returns comparison
fig, ax = plt.subplots(figsize=(10, 5))

(1 + rets).cumprod().plot(ax=ax, label="Momentum")
(1 + rsi_rets).cumprod().plot(ax=ax, label="RSI")
(1 + macd_rets).cumprod().plot(ax=ax, label="MACD")

ax.set_title("Cumulative Returns: Factor Comparison")
ax.set_ylabel("Growth of $1")
ax.legend()
plt.show()